<a href="https://colab.research.google.com/github/Roubish/NLP-Project/blob/main/Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
RESEARCH_DIR = "/content/drive/MyDrive/Roubish/Fake News Detection"
os.chdir(RESEARCH_DIR)
os.getcwd()

In [ ]:
!unzip "/content/drive/MyDrive/Roubish/Fake News Detection/fake-news.zip" -d "/content/drive/MyDrive/Roubish/Fake News Detection"

In [ ]:
import pandas as pd
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
# Vocabulary size
voc_size=5000

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Roubish/Fake News Detection/train.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#Drop Nan Values
df=df.dropna()

In [ ]:
# Get the Independent Features
X=df.drop('label',axis=1)

In [ ]:
# Get the Dependent features
y=df['label']

In [ ]:
y.value_counts()

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
# Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

In [ ]:
embedded_docs[1]

In [ ]:
# Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Adding Dropout

In [ ]:
# Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.5))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

In [ ]:
len(embedded_docs),y.shape

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,random_state=42)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training

In [ ]:
# Finally Training
train_model=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64) 


## Performance Metrics And Accuracy

In [ ]:
y_pred1=model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_model.history['accuracy'],'b',label='train_accuracy')
plt.plot(train_model.history['val_accuracy'],'r',label='val_accuracy')
plt.legend()

In [ ]:
# pred_val= model.predict_classes(x_val)
y_pred1=model.predict_classes(X_test)

from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

cm= confusion_matrix(y_test,y_pred1)
plot_confusion_matrix(cm, figsize=(5,5))